In [225]:
import   pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from nba_api.stats.endpoints import playergamelogs
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization

In [226]:
# 2022-23 Logs
game_logs = playergamelogs.PlayerGameLogs(
    season_nullable='2021-22'
)
data = game_logs.get_data_frames()[0]
data

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,VIDEO_AVAILABLE_FLAG
0,2021-22,1629680,Matisse Thybulle,Matisse,1610612755,PHI,Philadelphia 76ers,0022101228,2022-04-10T00:00:00,PHI vs. DET,...,1,22177,18239,14596,16281,11817,2151,131,13102,1
1,2021-22,203200,Justin Holiday,Justin,1610612758,SAC,Sacramento Kings,0022101229,2022-04-10T00:00:00,SAC @ PHX,...,17438,22177,11915,3883,17172,7727,2151,131,5470,1
2,2021-22,1630163,LaMelo Ball,LaMelo,1610612766,CHA,Charlotte Hornets,0022101217,2022-04-10T00:00:00,CHA vs. WAS,...,1,5520,4544,1936,1348,1833,1,131,686,1
3,2021-22,1630209,Omer Yurtseven,Omer,1610612748,MIA,Miami Heat,0022101227,2022-04-10T00:00:00,MIA @ ORL,...,1,5520,4544,14596,7652,9732,2151,131,10388,1
4,2021-22,1630215,Jared Butler,Jared,1610612762,UTA,Utah Jazz,0022101230,2022-04-10T00:00:00,UTA @ POR,...,1,11965,7362,9927,2414,14587,2151,131,13102,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26034,2021-22,201566,Russell Westbrook,Russell,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19T00:00:00,LAL vs. GSW,...,1,22177,11915,13258,25389,15373,2151,131,13837,1
26035,2021-22,1627780,Gary Payton II,Gary,1610612744,GSW,Golden State Warriors,0022100002,2021-10-19T00:00:00,GSW @ LAL,...,1,1,18239,23122,12132,25086,2151,131,25115,1
26036,2021-22,1628370,Malik Monk,Malik,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19T00:00:00,LAL vs. GSW,...,1,5520,18239,15707,21069,15293,2151,131,14542,1
26037,2021-22,1627761,DeAndre' Bembry,DeAndre',1610612751,BKN,Brooklyn Nets,0022100001,2021-10-19T00:00:00,BKN @ MIL,...,1,1,18239,23122,20351,25086,2151,131,25115,1


In [227]:
ohe = OneHotEncoder()
le = LabelEncoder()

data['MATCHUP'] = data['MATCHUP'].apply(lambda x: x.split()[2])

In [228]:
cols_cat = ['TEAM_ABBREVIATION', 'MATCHUP', 'PLAYER_NAME']
def encode_columns(dataset):
    export = {}
    for col in cols_cat:
        dataset[col] = le.fit_transform(dataset[col])
        enc_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        export[col] = enc_dict
    return export
le_dict = encode_columns(data)
le_dict

{'TEAM_ABBREVIATION': {'ATL': 0,
  'BKN': 1,
  'BOS': 2,
  'CHA': 3,
  'CHI': 4,
  'CLE': 5,
  'DAL': 6,
  'DEN': 7,
  'DET': 8,
  'GSW': 9,
  'HOU': 10,
  'IND': 11,
  'LAC': 12,
  'LAL': 13,
  'MEM': 14,
  'MIA': 15,
  'MIL': 16,
  'MIN': 17,
  'NOP': 18,
  'NYK': 19,
  'OKC': 20,
  'ORL': 21,
  'PHI': 22,
  'PHX': 23,
  'POR': 24,
  'SAC': 25,
  'SAS': 26,
  'TOR': 27,
  'UTA': 28,
  'WAS': 29},
 'MATCHUP': {'ATL': 0,
  'BKN': 1,
  'BOS': 2,
  'CHA': 3,
  'CHI': 4,
  'CLE': 5,
  'DAL': 6,
  'DEN': 7,
  'DET': 8,
  'GSW': 9,
  'HOU': 10,
  'IND': 11,
  'LAC': 12,
  'LAL': 13,
  'MEM': 14,
  'MIA': 15,
  'MIL': 16,
  'MIN': 17,
  'NOP': 18,
  'NYK': 19,
  'OKC': 20,
  'ORL': 21,
  'PHI': 22,
  'PHX': 23,
  'POR': 24,
  'SAC': 25,
  'SAS': 26,
  'TOR': 27,
  'UTA': 28,
  'WAS': 29},
 'PLAYER_NAME': {'Aaron Gordon': 0,
  'Aaron Henry': 1,
  'Aaron Holiday': 2,
  'Aaron Nesmith': 3,
  'Aaron Wiggins': 4,
  'Abdel Nader': 5,
  'Ade Murkey': 6,
  'Admiral Schofield': 7,
  'Ahmad Caver': 8,

In [229]:
X_cols_num = ['MIN', 'FGA', 'FG3A', 'FTA']
y_cols = ['PTS']

pts_X = data[X_cols_num]
pts_y = data[y_cols]

X_train_pts, X_test_pts, y_train_pts, y_test_pts = train_test_split(pts_X, pts_y, train_size=0.8)

# SVR Model (No Tuning)

In [230]:
from sklearn.svm import SVR
alpha = SVR(verbose=True).fit(X_train_pts, y_train_pts)

/Users/iyinoluwatugbobo/Library/Application Support/JetBrains/DataSpell2022.3/projects/workspace/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[LibSVM]

In [231]:
alpha_pred_pts = alpha.predict(X_test_pts)
print(f"MSE: {mean_squared_error(y_test_pts, alpha_pred_pts)}")
print(f"R2: {r2_score(y_test_pts, alpha_pred_pts)}")

MSE: 12.51200917547805
R2: 0.8379015183739547


# SVR Models (w/ Tuning)

In [235]:
beta = SVR(
    kernel='rbf', C=1000, epsilon=1
).fit(X_train_pts, y_train_pts)

/Users/iyinoluwatugbobo/Library/Application Support/JetBrains/DataSpell2022.3/projects/workspace/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [236]:
beta_pred_pts = beta.predict(X_test_pts)
print(f"MSE: {mean_squared_error(y_test_pts, beta_pred_pts)}")
print(f"R2: {r2_score(y_test_pts, beta_pred_pts)}")

MSE: 12.32370799115103
R2: 0.8403410415264486


In [234]:
player_key = le_dict['PLAYER_NAME']
team_key = le_dict['TEAM_ABBREVIATION']

beta.predict([[
    37,
    21.4,
    9.3,
    8.5
]])

array([30.35509605])

In [238]:
chi = SVR(
    kernel='rbf', C=100, epsilon=1
).fit(X_train_pts, y_train_pts)

/Users/iyinoluwatugbobo/Library/Application Support/JetBrains/DataSpell2022.3/projects/workspace/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [239]:
chi_pred_pts = chi.predict(X_test_pts)
print(f"MSE: {mean_squared_error(y_test_pts, chi_pred_pts)}")
print(f"R2: {r2_score(y_test_pts, chi_pred_pts)}")

MSE: 12.237945771545863
R2: 0.8414521281140546


In [250]:
for i in range(1, 101):
    model = SVR(
        C=i,
        epsilon=1
    ).fit(X_train_pts, y_train_pts['PTS'])
    proj = model.predict(X_test_pts)
    mse = mean_squared_error(y_test_pts, proj)
    r2 = r2_score(y_test_pts, proj)
    print(f"C={i}\nMSE -> {round(mse, 5)}\nR2 -> {round(r2, 5)}")


C=1
MSE -> 12.51983
R2 -> 0.8378
C=2
MSE -> 12.40418
R2 -> 0.8393
C=3
MSE -> 12.35329
R2 -> 0.83996
C=4
MSE -> 12.31908
R2 -> 0.8404
C=5
MSE -> 12.29699
R2 -> 0.84069
C=6
MSE -> 12.28012
R2 -> 0.84091
C=7
MSE -> 12.27526
R2 -> 0.84097
C=8
MSE -> 12.27007
R2 -> 0.84104
C=9
MSE -> 12.26616
R2 -> 0.84109
C=10
MSE -> 12.25529
R2 -> 0.84123
C=11
MSE -> 12.24945
R2 -> 0.8413
C=12
MSE -> 12.24427
R2 -> 0.84137
C=13
MSE -> 12.23903
R2 -> 0.84144
C=14
MSE -> 12.23848
R2 -> 0.84145
C=15
MSE -> 12.23839
R2 -> 0.84145
C=16
MSE -> 12.23765
R2 -> 0.84146
C=17
MSE -> 12.23716
R2 -> 0.84146
C=18
MSE -> 12.23393
R2 -> 0.8415
C=19
MSE -> 12.23282
R2 -> 0.84152
C=20
MSE -> 12.2327
R2 -> 0.84152
C=21
MSE -> 12.23183
R2 -> 0.84153
C=22
MSE -> 12.22957
R2 -> 0.84156
C=23
MSE -> 12.22899
R2 -> 0.84157
C=24
MSE -> 12.22964
R2 -> 0.84156
C=25
MSE -> 12.23116
R2 -> 0.84154
C=26
MSE -> 12.23192
R2 -> 0.84153
C=27
MSE -> 12.23169
R2 -> 0.84153
C=28
MSE -> 12.23058
R2 -> 0.84155
C=29
MSE -> 12.23087
R2 -> 0.84154
